In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import sys
import pandas as pd
#sys.path.append("/home/CVShare/Jeni/hts/machine_learning/sampling_with_data_cleaning")
import sampling_with_data_cleaning as sdc
from utils import *

### FNN on cleaned features

In [2]:
findex = 4
x,y = get_features(findex,train=True,cleaned=True)
print y.shape
print sum(y)
y = y.astype(int)
print sum(y)
x,y = shuffle(x,y)

(47755,)
266.0
266


In [3]:
no_ex,ip_dim = x.shape
print no_ex,ip_dim

47755 155


In [4]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,500)
        self.l2 = nn.Linear(500,500)
        self.l3 = nn.Linear(500,250)
        self.l4 = nn.Linear(250,50)
        self.l5 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        x = F.leaky_relu(self.l4(x))
        x = (self.l5(x))
        
        return x

In [6]:
max_percent = 0.4

for i in range(1,6):
    val_iter = i
    print("val iter: ",val_iter)
    
   
    #weights_array = [7]
    weights_array = np.linspace(20,50,10)
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(2000):
            
            ind = get_train_ind(val_iter=val_iter,no_examples=no_ex)
            xtrain,ytrain = get_train_batch(x,y,batch_size=50,indices=ind)
            
            model_op = mymlp(xtrain)
           
            loss = criterion(model_op,ytrain)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    #     if(ep%30==29):
    #         print(loss.data[0])

        ## After training check on cross validation data
        xval,yval = get_val_data(x,y,no_examples=no_ex,val_iter=val_iter)
        min_fp = xval.size()[0]*0.2
        yval = yval.reshape(yval.shape[0],)
        train_op = mymlp(xval)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
      
        cf = metrics.confusion_matrix(yval,pred_labels).ravel()
        [tn,fp,fn,tp] = cf
        print('tn, fp, fn, tp: ',cf)
        percent = float(float(tp)/float(tp+fn))
        if(percent>max_percent):
            if(1==1):
                if(fp < min_fp):
                    min_fp = fp
                    max_percent = percent
                    print("min fp, max_percent",fp,percent)
                    model_path = os.getcwd() + '/fnn_clean' + str(findex)
                    torch.save(mymlp.state_dict(),model_path)
                    print("saving model on val: ",val_iter," and weight: ",w)

        


('val iter: ', 1)
('tn, fp, fn, tp: ', array([9403,   99,   38,   11]))
('tn, fp, fn, tp: ', array([9406,   96,   35,   14]))
('tn, fp, fn, tp: ', array([9434,   68,   36,   13]))
('tn, fp, fn, tp: ', array([9305,  197,   30,   19]))
('tn, fp, fn, tp: ', array([9336,  166,   30,   19]))
('tn, fp, fn, tp: ', array([9195,  307,   24,   25]))
('min fp, max_percent', 307, 0.5102040816326531)
('saving model on val: ', 1, ' and weight: ', 36.666666666666671)
('tn, fp, fn, tp: ', array([9206,  296,   27,   22]))
('tn, fp, fn, tp: ', array([9305,  197,   32,   17]))
('tn, fp, fn, tp: ', array([9271,  231,   28,   21]))
('tn, fp, fn, tp: ', array([9373,  129,   34,   15]))
('val iter: ', 2)
('tn, fp, fn, tp: ', array([9475,   17,   49,   10]))
('tn, fp, fn, tp: ', array([9483,    9,   52,    7]))
('tn, fp, fn, tp: ', array([9397,   95,   38,   21]))
('tn, fp, fn, tp: ', array([9381,  111,   33,   26]))
('tn, fp, fn, tp: ', array([9407,   85,   29,   30]))
('tn, fp, fn, tp: ', array([9331,  161,

KeyboardInterrupt: 

In [7]:
xtest,ytest = get_features(findex,train=False)
xtest = Variable(torch.cuda.FloatTensor(xtest).cuda())
test_model = c_mlp().cuda()
test_model.load_state_dict(torch.load(model_path))
test_op = test_model(xtest)
print type(test_op)

<class 'torch.autograd.variable.Variable'>


In [8]:
test_op = test_op.cpu().data.numpy()
pred_labels = np.argmax(test_op,axis=1)
cf = metrics.confusion_matrix(ytest,pred_labels).ravel()
print('tn, fp, fn, tp: ',cf)

('tn, fp, fn, tp: ', array([11734,   211,     8,     4]))
